# check Techaer accuracy: 3C

In [1]:
from good_tacher_resnet50 import val_transform, FilteredLymphoMNIST, get_dataloaders
from good_tacher_resnet50 import WeightedRandomSampler, balanced_weights
from LymphoMNIST.LymphoMNIST import LymphoMNIST

LymphoMNIST v0.0.1 @ https://github.com/Khayrulbuet13/Lympho3-MNIST.git


In [2]:
# our hyperparameters
params = {
    'lr': 1e-5,
    'batch_size': 16,
    'epochs': 10000,
    'model': "Teacher_final-1c"
}


In [3]:

# Initialize dataset
original_train_ds = LymphoMNIST(root='./dataset', train=True, download=True, transform=val_transform, num_classes=3)
original_test_ds = LymphoMNIST(root='./dataset', train=False, download=True, transform=val_transform, num_classes=3)


# Specify labels to keep
labels_to_keep = [0, 1] # 0: B, 1: T4, 2: T8

# Initialize filtered dataset with labels to keep
train_ds = FilteredLymphoMNIST(original_train_ds, labels_to_keep)
test_ds= FilteredLymphoMNIST(original_test_ds, labels_to_keep)

weights = balanced_weights(train_ds, len(labels_to_keep))
sampler = WeightedRandomSampler(weights, len(weights))
# Create the dataloaders
train_dl, val_dl, test_dl = get_dataloaders(train_ds,
                                            test_ds,
                                            split=(0.5, 0.5),
                                            batch_size=params['batch_size'],
                                            sampler=sampler,
                                            num_workers=4
                                           )

Dataset already exists. Skipping download.
Dataset already exists. Skipping download.


In [4]:
import torch
from torchvision import models
from torch import nn

def calculate_accuracy(loader, model, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return accuracy



device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Load the model
model = models.resnet50()
model.fc = nn.Linear(model.fc.in_features, len(labels_to_keep))

# Load the saved weights
model.load_state_dict(torch.load("checkpoint/Final_models/28 September 02:53_Teacher_final-control_final_weights.pt"))
model = model.to(device)


In [5]:
test_acc = calculate_accuracy(test_dl, model, device)
print(f'Test Accuracy: {test_acc:.2f}%')


Test Accuracy: 97.14%


In [6]:
test_acc = calculate_accuracy(train_dl, model, device)
print(f'Test Accuracy: {test_acc:.2f}%')


Test Accuracy: 99.30%


In [7]:
test_acc = calculate_accuracy(val_dl, model, device)
print(f'Test Accuracy: {test_acc:.2f}%')

Test Accuracy: 97.55%


# check Techaer accuracy: 1C

In [ ]:
from good_tacher_resnet50 import val_transform, FilteredLymphoMNIST, get_dataloaders
from good_tacher_resnet50 import WeightedRandomSampler, balanced_weights
from LymphoMNIST.LymphoMNIST import LymphoMNIST

In [ ]:
# our hyperparameters
params = {
    'lr': 1e-5,
    'batch_size': 16,
    'epochs': 10000,
    'model': "Teacher_final-1c"
}

In [ ]:

# Initialize dataset
original_train_ds = LymphoMNIST(root='./dataset', train=True, download=True, transform=val_transform, num_classes=3)
original_test_ds = LymphoMNIST(root='./dataset', train=False, download=True, transform=val_transform, num_classes=3)


# Specify labels to keep
labels_to_keep = [0, 1] # 0: B, 1: T4, 2: T8

# Initialize filtered dataset with labels to keep
train_ds = FilteredLymphoMNIST(original_train_ds, labels_to_keep)
test_ds= FilteredLymphoMNIST(original_test_ds, labels_to_keep)

weights = balanced_weights(train_ds, len(labels_to_keep))
sampler = WeightedRandomSampler(weights, len(weights))
# Create the dataloaders
train_dl, val_dl, test_dl = get_dataloaders(train_ds,
                                            test_ds,
                                            split=(0.5, 0.5),
                                            batch_size=params['batch_size'],
                                            sampler=sampler,
                                            num_workers=4
                                           )

In [ ]:
import torch
from torchvision import models
from torch import nn

def calculate_accuracy(loader, model, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return accuracy



device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Load pre-trained ResNet50 model
model = models.resnet50(weights=None)
num_ftrs = model.fc.in_features
num_classes = len(labels_to_keep)
model.fc = nn.Linear(num_ftrs, num_classes)

# Load the saved weights
model.load_state_dict(torch.load("checkpoint/Final_models/28 September 02:53_Teacher_final-control_final_weights.pt"))
model = model.to(device)


In [ ]:
test_acc = calculate_accuracy(test_dl, model, device)
print(f'Test Accuracy: {test_acc:.2f}%')


In [ ]:
test_acc = calculate_accuracy(train_dl, model, device)
print(f'Test Accuracy: {test_acc:.2f}%')


In [ ]:
test_acc = calculate_accuracy(val_dl, model, device)
print(f'Test Accuracy: {test_acc:.2f}%')

# check Student accuracy: qt

In [1]:
from student_qt import transform_student, FilteredLymphoMNIST, get_dataloaders
# from student_qt import WeightedRandomSampler, balanced_weights
from LymphoMNIST.LymphoMNIST import LymphoMNIST

In [4]:
# our hyperparameters
EPOCHS = 100000
TEMPERATURE = 1
INIT_LR = 0.001
WEIGHT_DECAY = .0001
CLIP_THRESHOLD = 1.0
ALPHA = 1
BATCH_SIZE = 64
RESIZE = 64
BIGGER = 64
MODEL = 'resnet50_qt-1channel'

In [2]:



# Specify labels to keep
labels_to_keep = [0, 1] # 0: B, 1: T4, 2: T8

# Initialize dataset
train_ds_student = FilteredLymphoMNIST(
        LymphoMNIST(root='./dataset', train=True, download=True, transform=transform_student, num_classes=3), labels_to_keep)
val_test_ds_student = FilteredLymphoMNIST(
        LymphoMNIST(root='./dataset', train=False, download=True, transform=transform_student, num_classes=3), labels_to_keep)


# Initialize filtered dataset with labels to keep
train_ds = FilteredLymphoMNIST(train_ds_student, labels_to_keep)
test_ds= FilteredLymphoMNIST(val_test_ds_student, labels_to_keep)

# Create the dataloaders
train_dl, val_dl, test_dl = get_dataloaders(train_ds_student, val_test_ds_student, num_workers=4)


Dataset already exists. Skipping download.
Dataset already exists. Skipping download.


In [5]:
import torch
from torchvision import models
from torch import nn

def calculate_accuracy(loader, model, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return accuracy



device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Load pre-trained ResNet50 model
class QuantizedCNN(nn.Module):
    def __init__(self, num_classes=2, input_size=(1, 28, 28)):
        super(QuantizedCNN, self).__init__()
        self.num_classes = num_classes
        self.features = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0),
            nn.Conv2d(16, 16, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        )
        with torch.no_grad():
            dummy_input = torch.zeros(1, *input_size)
            dummy_output = self.features(dummy_input)
            num_ftrs = dummy_output.numel() // dummy_output.size(0)
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Dropout(0.5),
            nn.Linear(num_ftrs, num_classes),
            nn.Softmax(dim=1)
        )
    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x
    
    
num_classes = len(labels_to_keep)
model = QuantizedCNN(num_classes=num_classes, input_size=(1, BIGGER, BIGGER)).to(device)

# Load the saved weights
model.load_state_dict(torch.load("checkpoint/Final_models/KD_30 September 13:05_resnet50_qt-1channel.pt"))
model = model.to(device)


In [6]:
test_acc = calculate_accuracy(test_dl, model, device)
print(f'Test Accuracy: {test_acc:.2f}%')


Test Accuracy: 90.71%


In [9]:
test_acc = calculate_accuracy(train_dl, model, device)
print(f'Train Accuracy: {test_acc:.2f}%')

Train Accuracy: 92.16%


In [10]:
test_acc = calculate_accuracy(val_dl, model, device)
print(f'Val Accuracy: {test_acc:.2f}%')

Val Accuracy: 89.98%
